# Phân Tích Yêu Cầu Hoàn Trả Chi Phí

Notebook này minh họa cách tạo các agent sử dụng plugin để xử lý chi phí đi lại từ hình ảnh hóa đơn địa phương, tạo email yêu cầu hoàn trả chi phí, và trực quan hóa dữ liệu chi phí bằng biểu đồ tròn. Các agent sẽ tự động chọn chức năng dựa trên ngữ cảnh của nhiệm vụ.

Các bước:
1. OCR Agent xử lý hình ảnh hóa đơn địa phương và trích xuất dữ liệu chi phí đi lại.
2. Email Agent tạo email yêu cầu hoàn trả chi phí.

### Ví dụ về một kịch bản chi phí đi lại:
Hãy tưởng tượng bạn là một nhân viên đi công tác để tham dự một cuộc họp kinh doanh ở một thành phố khác. Công ty của bạn có chính sách hoàn trả tất cả các chi phí liên quan đến đi lại hợp lý. Dưới đây là phân loại các chi phí đi lại có thể phát sinh:
- **Phương tiện di chuyển:**
Vé máy bay khứ hồi từ thành phố nơi bạn sinh sống đến thành phố điểm đến.  
Dịch vụ taxi hoặc xe công nghệ để di chuyển đến và từ sân bay.  
Phương tiện di chuyển tại thành phố điểm đến (như phương tiện công cộng, thuê xe, hoặc taxi).

- **Chỗ ở:**
Lưu trú tại khách sạn trong ba đêm tại một khách sạn tầm trung gần địa điểm họp.

- **Bữa ăn:**
Phụ cấp bữa ăn hàng ngày cho bữa sáng, bữa trưa và bữa tối, dựa trên chính sách chi tiêu của công ty.

- **Chi phí khác:**
Phí đỗ xe tại sân bay.  
Phí truy cập Internet tại khách sạn.  
Tiền tip hoặc các khoản phí dịch vụ nhỏ.

- **Tài liệu:**
Bạn nộp tất cả các hóa đơn (vé máy bay, taxi, khách sạn, bữa ăn, v.v.) và một báo cáo chi phí đã hoàn thành để được hoàn trả.


## Nhập các thư viện cần thiết

Nhập các thư viện và module cần thiết cho notebook.


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## Định nghĩa Các Mô Hình Chi Tiêu

Tạo một mô hình Pydantic cho từng khoản chi tiêu và một lớp ExpenseFormatter để chuyển đổi truy vấn của người dùng thành dữ liệu chi tiêu có cấu trúc.

Mỗi khoản chi tiêu sẽ được biểu diễn theo định dạng:  
`{'date': '07-Mar-2025', 'description': 'chuyến bay đến điểm đến', 'amount': 675.99, 'category': 'Vận chuyển'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## Định nghĩa Agents - Tạo Email

Tạo một lớp agent để tạo email gửi yêu cầu bồi hoàn chi phí.  
- Agent này sử dụng trình trang trí `kernel_function` để định nghĩa một hàm tạo email gửi yêu cầu bồi hoàn chi phí.  
- Nó tính tổng số tiền của các chi phí và định dạng chi tiết vào nội dung email.  


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# Đại lý trích xuất chi phí du lịch từ hình ảnh hóa đơn

Tạo một lớp đại lý để trích xuất chi phí du lịch từ hình ảnh hóa đơn.  
- Đại lý này sử dụng trình trang trí `kernel_function` để định nghĩa một hàm trích xuất chi phí du lịch từ hình ảnh hóa đơn.  
- Chuyển đổi hình ảnh hóa đơn thành văn bản bằng OCR (Nhận dạng ký tự quang học) và trích xuất thông tin liên quan như ngày tháng, mô tả, số tiền, và danh mục.  


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## Xử lý Chi Phí

Định nghĩa một hàm bất đồng bộ để xử lý chi phí bằng cách tạo và đăng ký các tác nhân cần thiết, sau đó kích hoạt chúng.
- Hàm này xử lý chi phí bằng cách tải các biến môi trường, tạo các tác nhân cần thiết và đăng ký chúng dưới dạng plugin.
- Nó tạo một cuộc trò chuyện nhóm với hai tác nhân và gửi một tin nhắn nhắc để tạo email và biểu đồ hình tròn dựa trên dữ liệu chi phí.
- Nó xử lý bất kỳ lỗi nào xảy ra trong quá trình kích hoạt cuộc trò chuyện và đảm bảo dọn dẹp đúng cách các tác nhân.


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## Hàm chính

Định nghĩa hàm chính để xóa màn hình console và chạy hàm `process_expenses` một cách bất đồng bộ.


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
